# Common Code for Experiments

This notebook contains common code that is used in the various causal experiments done in AitiaExplorer.

## Contents

TBD

## References

Portions of the code dealing with selecting the optimal number of clusters is based on the article and code provided at:


https://towardsdatascience.com/gaussian-mixture-model-clusterization-how-to-select-the-number-of-components-clusters-553bef45f6e4

In [ ]:
# imports

import os
import sys
import pandas as pd
import networkx as nx
import numpy as np
from sklearn import metrics
from sklearn import mixture
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.model_selection import train_test_split
from pycausal.pycausal import pycausal

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aitia_explorer.app import App

# stop the warning clutter
import warnings
warnings.filterwarnings('ignore')

## Utility Functions

In [ ]:
def select_best(arr, x):
    """
    returns the set of x configurations with shorter distance.
    """
    dx = np.argsort(arr)[:x]
    return arr[dx]


def draw_ellipse(position, covariance, ax=None, **kwargs):
    """
    Draw an ellipse with a given position and covariance.
    """
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        

def plot_gmm(gmm, X, label=True, ax=None):
    """
    Plot a GMM.
    """
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)

    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)
        
    plt.title("GMM with %d components" % len(gmm.means_), fontsize=(20))
    plt.xlabel("U.A.")
    plt.ylabel("U.A.")

## BIC Score Functions

In [ ]:
def get_bic_scores(min_clusters, max_clusters, df):
    n_clusters = np.arange(min_clusters, max_clusters)
    bics = []
    bics_err = []
    iterations = max_clusters
    for n in n_clusters:
        tmp_bic = []
        for _ in range(iterations):
            gmm = mixture.GaussianMixture(n, n_init=2).fit(df)
            tmp_bic.append(gmm.bic(df))
        val = np.mean(select_best(np.array(tmp_bic), int(iterations / 5)))
        err = np.std(tmp_bic)
        bics.append(val)
        bics_err.append(err)
    return bics, bics_err

In [ ]:
def plot_bic_aic_scores(bics, bics_err, min_clusters, max_clusters):
    n_clusters = np.arange(min_clusters, max_clusters)
    plt.errorbar(n_clusters, bics, yerr=bics_err, label='BIC')
    plt.title("BIC and AIC Scores", fontsize=20)
    plt.xticks(n_clusters)
    plt.xlabel("N. of clusters")
    plt.ylabel("Score")
    plt.legend()
    plt.show()

In [ ]:
def plot_bic_gradient(bics, bics_err, min_clusters, max_cluster):
    n_clusters = np.arange(min_clusters, max_clusters)
    plt.errorbar(n_clusters, np.gradient(bics), yerr=bics_err, label='BIC')
    plt.title("Gradient of BIC Scores", fontsize=20)
    plt.xticks(n_clusters)
    plt.xlabel("N. of clusters")
    plt.ylabel("grad(BIC)")
    plt.legend()
    plt.show()

## Train / Test Distance Functions

In [ ]:
# As provided at https://stackoverflow.com/questions/26079881/kl-divergence-of-two-gmms. 

def gmm_js(gmm_p, gmm_q, n_samples=10 ** 5):
    X = gmm_p.sample(n_samples)[0]
    log_p_X = gmm_p.score_samples(X)
    log_q_X = gmm_q.score_samples(X)
    log_mix_X = np.logaddexp(log_p_X, log_q_X)

    Y = gmm_q.sample(n_samples)[0]
    log_p_Y = gmm_p.score_samples(Y)
    log_q_Y = gmm_q.score_samples(Y)
    log_mix_Y = np.logaddexp(log_p_Y, log_q_Y)

    return np.sqrt((log_p_X.mean() - (log_mix_X.mean() - np.log(2))
                    + log_q_Y.mean() - (log_mix_Y.mean() - np.log(2))) / 2)


In [ ]:
def get_train_test_distance(min_clusters, max_clusters, df):
    n_clusters = np.arange(min_clusters, max_clusters)
    iterations = max_clusters
    results = []
    res_sigs = []
    for n in n_clusters:
        dist = []
        for iteration in range(iterations):
            train, test = train_test_split(df, test_size=0.5)
            gmm_train = mixture.GaussianMixture(n, n_init=2).fit(train)
            gmm_test = mixture.GaussianMixture(n, n_init=2).fit(test)
            dist.append(gmm_js(gmm_train, gmm_test))
        selec = select_best(np.array(dist), int(iterations / 5))
        result = np.mean(selec)
        res_sig = np.std(selec)
        results.append(result)
        res_sigs.append(res_sig)
    return results, res_sigs

In [ ]:
def plot_train_test_distance(results, res_sigs, min_clusters, max_cluster):
    n_clusters = np.arange(min_clusters, max_clusters)
    plt.errorbar(n_clusters, results, yerr=res_sigs)
    plt.title("Distance between Train and Test GMMs", fontsize=20)
    plt.xticks(n_clusters)
    plt.xlabel("N. of clusters")
    plt.ylabel("Distance")
    plt.show()

## Silhouette Coefficient Functions

In [ ]:
def plot_train_test_distance(results, res_sigs, min_clusters, max_clusters):
    n_clusters = np.arange(min_clusters, max_clusters)
    plt.errorbar(n_clusters, results, yerr=res_sigs)
    plt.title("Distance between Train and Test GMMs", fontsize=20)
    plt.xticks(n_clusters)
    plt.xlabel("N. of clusters")
    plt.ylabel("Distance")
    plt.show()

In [ ]:
def get_silhouette_coefficient(min_clusters, max_clusters, df):
    n_clusters = np.arange(min_clusters, max_clusters)
    sils = []
    sils_err = []
    iterations = max_clusters
    for n in n_clusters:
        tmp_sil = []
        for _ in range(iterations):
            gmm = mixture.GaussianMixture(n, n_init=2).fit(df)
            labels = gmm.predict(df)
            sil = metrics.silhouette_score(df, labels, metric='euclidean')
            tmp_sil.append(sil)
        val = np.mean(select_best(np.array(tmp_sil), int(iterations / 5)))
        err = np.std(tmp_sil)
        sils.append(val)
        sils_err.append(err)
    return sils, sils_err

In [ ]:
def plot_silhouette_coefficient(sils, sils_err, min_clusters, max_clusters):
    n_clusters = np.arange(min_clusters, max_clusters)
    plt.errorbar(n_clusters, sils, yerr=sils_err)
    plt.title("Silhouette Scores", fontsize=20)
    plt.xticks(n_clusters)
    plt.xlabel("N. of clusters")
    plt.ylabel("Score")
    plt.show()